# 对象获取和借展

> **上级页面**: ["流传历史"]("/model/provenance/")

---



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 设置 base_url 以使输出更清晰
model.factory.base_url = 'http://test.com/museum/'




## 简介

大多数记录的流传历史事件是对象所有权变更时，以及发生的各种方式。在下面的描述中，"卖方"和"买方"术语用于指代放弃所有权的行动者和获得所有权的行动者，即使没有金钱或其他商品或服务交换。这是为了描述的清晰和简洁，而不是暗示每笔交易都是购买。

## 对象获取

获取用于描述对象所有权从一个所有者转移到下一个所有者。第一个所有者通常是艺术家，然后将其转移给第二个所有者，再转移给第三个所有者，依此类推。所有权链可以通过重复相同的模式来表达，一次获取中的买方成为后续获取中的卖方。如果特定获取的前一个所有者（例如卖方）或后一个所有者（例如买方）未知，则可以从描述中省略该引用。

获取不一定是购买，它可能是赠与、继承或任何其他获得对象所有权的方法。

The model encodes this information with an `Acquisition` part of the overall Provenance Event. The acquisition is the transfer of the right of ownership of an object (referenced in `transferred_title_of`) from the seller (in `transferred_title_from`) to the buyer (in `transferred_title_to`). 

Each object has its own Acquisition as part of the provenance event, so if a collector buys three paintings from a dealer, then there would be a single Provenance Event with three Acquisitions, all of which transfer the title of a single painting from the dealer to the collector.


__Example:__

Oliver Payne acquired Spring from the Durand-Ruel Gallery in New York in 1909



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (venance entry for the purchase)
# 行为者: Oliver Payne (buyer), Durand-Ruel Gallery (seller)
# 对象: ProvenanceEntry representing the purchase of "Spring" painting
# 目的: To document the transfer of ownership from gallery to collector
provenance_entry = vocab.ProvenanceEntry(
    ident="durand_payne/1",
    label="Purchase of Spring by Payne"
)
provenance_entry.identified_by = vocab.PrimaryName(
    content="Purchase of Spring by Payne from Durand-Ruel"
)

# 步骤 4: 创建相关对象和关系
# When: The purchase occurred in 1909
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "1909-01-01T00:00:00Z"
timespan.end_of_the_end = "1909-12-31T23:59:59Z"
provenance_entry.timespan = timespan

# Acquisition event - transfer of ownership
acquisition = model.Acquisition(label="Ownership of Spring to Payne")
provenance_entry.part = acquisition

# What object was transferred: The "Spring" painting
acquisition.transferred_title_of = model.HumanMadeObject(
    ident="spring",
    label="Spring"
)

# Who transferred ownership from: Durand-Ruel Gallery (seller)
acquisition.transferred_title_from = model.Person(
    ident="durand",
    label="Durand-Ruel Gallery"
)

# Who transferred ownership to: Oliver Payne (buyer)
acquisition.transferred_title_to = model.Person(
    ident="payne",
    label="Oliver Payne"
)

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(provenance_entry, compact=False))



### Multiple Owners

In the simple case of multiple simultaneous owners where either the division of the ownership is unknown or an even split, all of the owners can simply be listed with `transferred_title_from` (when the group is selling) or `transferred_title_to` (when the group is buying).  

If there is additional information known about the exact nature of the ownership division, then it is worth considering the more complex and more expressive section on [rights](../rights) to describe this in more detail.

If there are multiple parties that have formally entered into a legal consortium or organization, and that consortium is the legal owner of the object, then the consortium should be modeled as a [`Group`](/model/actor/) with the parties as members, and be the sole owner of the object.

__Example:__

The Yale University Art Gallery and the Yale Center for British Art jointly purchased a painting by Kehinde Wiley in 2021 from an undisclosed seller.



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (venance entry for joint purchase)
# 行为者: Yale University Art Gallery and Yale Center for British Art (buyers)
# 对象: ProvenanceEntry representing a joint purchase
# 目的: To document multiple owners acquiring the same artwork
provenance_entry = vocab.ProvenanceEntry(
    ident="yuag_ycba_wiley/1",
    label="Purchase of Painting"
)

# 步骤 4: 创建相关对象和关系
# When: The joint purchase occurred in 2021
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "2021-01-01T00:00:00Z"
timespan.end_of_the_end = "2021-12-31T23:59:59Z"
provenance_entry.timespan = timespan

# Acquisition event
acquisition = model.Acquisition()
provenance_entry.part = acquisition

# What object was transferred: The painting
acquisition.transferred_title_of = model.HumanMadeObject(
    ident="yiadom-boakye",
    label="Portrait of Lynette Yiadom-Boakye"
)

# Who transferred ownership to: Multiple institutions (joint owners)
acquisition.transferred_title_to = model.Group(
    ident="yuag",
    label="Yale University Art Gallery"
)
acquisition.transferred_title_to = model.Group(
    ident="ycba",
    label="Yale Center for British Art"
)

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(provenance_entry, compact=False))



### Agents

In some cases, it is known that an agent other than the buyer facilitated or carried out the acquisitionon the buyer's behalf. This can be modeled by associating a different person or group with the activity using `carried_out_by` as the relationship. Multiple agents can be listed in this way, but it is not possible to associate agents with the person or group they're working for -- the model does not have a notion of seller's agent as distinct from buyer's agent.

__Example:__

In 2014, the family heirs of Oliver Payne sold Spring (by auction at Christie's) to Otto Naumann, acting for the J. Paul Getty Museum.



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (venance entry for auction purchase)
# 行为者: J. Paul Getty Museum (buyer), Family of Oliver Payne (seller), Otto Naumann (agent)
# 对象: ProvenanceEntry representing auction purchase via agent
# 目的: To document acquisition facilitated by an agent acting on behalf of the buyer
provenance_entry = vocab.ProvenanceEntry(
    ident="payneheirs_getty/1",
    label="Purchase of Spring for Getty"
)

# 步骤 4: 创建相关对象和关系
# When: The auction purchase occurred on November 5, 2014
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "2014-11-05T00:00:00Z"
timespan.end_of_the_end = "2014-11-05T23:59:59Z"
provenance_entry.timespan = timespan

# Acquisition event
acquisition = model.Acquisition(label="Acquisition of Spring")
provenance_entry.part = acquisition

# What object was transferred: The "Spring" painting
acquisition.transferred_title_of = model.HumanMadeObject(
    ident="spring",
    label="Spring"
)

# Who transferred ownership from: Family of Oliver Payne (sellers)
acquisition.transferred_title_from = model.Group(
    ident="payne_heirs",
    label="Family of Oliver Payne"
)

# Who transferred ownership to: J. Paul Getty Museum (buyer)
acquisition.transferred_title_to = model.Group(
    ident="getty",
    label="J. Paul Getty Museum"
)

# Who carried out the acquisition: Otto Naumann (agent acting for Getty)
acquisition.carried_out_by = model.Person(
    ident="naumann",
    label="Otto Naumann"
)

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(provenance_entry, compact=False))



### Exchange of Objects

This pattern allows for the exchange of objects between two parties by simply adding a second Acquisition with buyer and seller reversed.

__Example:__

In 1962, the Yale University Art Gallery exchanged "Nude Woman" by Beckmann for "Personnages dans un parc" by Masson.



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (venance entry for exchange)
# 行为者: Yale University Art Gallery and Richard Feigen Gallery (both buyers and sellers)
# 对象: ProvenanceEntry representing an exchange of artworks
# 目的: To document two parties trading artworks with each other
provenance_entry = vocab.ProvenanceEntry(
    ident="yuag_stora/1",
    label="Exchange of Objects"
)

# 步骤 4: 创建相关对象和关系
# 行为者: The two parties involved
yuag = model.Group(
    ident="yuag",
    label="Yale University Art Gallery"
)
feigen = model.Group(
    ident="feigen",
    label="Richard Feigen Gallery"
)

# First acquisition: YuAG acquires "Personnages dans un parc"
acquisition1 = model.Acquisition(label="Acquisition of Personnages")
provenance_entry.part = acquisition1

acquisition1.transferred_title_of = model.HumanMadeObject(
    ident="masson_personnages",
    label="Personnages dans un parc"
)
acquisition1.transferred_title_from = feigen  # Feigen sells
acquisition1.transferred_title_to = yuag      # YuAG buys

# Second acquisition: Feigen acquires "Nude Woman by Beckmann"
acquisition2 = model.Acquisition(label="Acquisition of Nude")
provenance_entry.part = acquisition2

acquisition2.transferred_title_of = model.HumanMadeObject(
    ident="beckmann_nude",
    label="Nude Woman by Beckmann"
)
acquisition2.transferred_title_from = yuag      # YuAG sells
acquisition2.transferred_title_to = feigen     # Feigen buys

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(provenance_entry, compact=False))




## Payments

A purchase is a common type of acquisition in which money is exchanged for the object.  The provenance event consists of the acquisition along with one or more related payments.  This typically involves a payment from the seller to the buyer for the agreed upon price, but might include further payments to or from others such as for shared ownership, payment of debts owed, or for services rendered as part of the overall activity.

The `Payment` activity has equivalent relationships for the actor that the money is transferred from (`paid_from`), the actor the money is transferred to (`paid_to`), and the amount of money paid (`paid_amount`).  The amount itself is a `MonetaryAmount` that has a value (`value`) and a currency (`currency`).  Each separable monetary amount is modeled as an individual instance of `Payment`, and thus if the buyer paid a commission to the agent who carried out the purchase, and paid for the object, then there would be two Payments, one from the buyer to the seller and one from the buyer to the agent. Commissions to auction houses or for consignments would use the same pattern.

!!! note "Diachronic Comparison of Monetary Amounts"
    The underlying ontology has specified that the `MonetaryAmount` refers to the face value of the combination of value and currency. This means that any comparison between `MonetaryAmount` instances should also take into account the datetimes of resources that reference it, rather than standing alone. It is unclear if the same `MonetaryAmount` can be used for all occurences of value and currency, or whether it is something more unique than that, and to be safe the model assumes that one instance will only be used in a single `Payment`.

__Example:__

Édouard Manet sold his painting, Jeanne, to Antonin Proust in 1883 for 3,000 francs.



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (venance entry for sale)
# 行为者: Antonin Proust (buyer), 爱德华·马奈 (seller)
# 对象: ProvenanceEntry representing a sale with payment
# 目的: To document both the transfer of ownership and the payment amount
provenance_entry = vocab.ProvenanceEntry(
    ident="manet_proust/1",
    label="Purchase of Spring by Proust"
)
provenance_entry.identified_by = vocab.PrimaryName(
    content="Purchase of Spring by Proust from Manet"
)

# 步骤 4: 创建相关对象和关系
# When: The sale occurred between 1881-1883
timespan = model.TimeSpan()
timespan.begin_of_the_begin = "1881-01-01T00:00:00Z"
timespan.end_of_the_end = "1883-12-31T23:59:59Z"
provenance_entry.timespan = timespan

# Acquisition event - transfer of ownership
acquisition = model.Acquisition(label="Ownership of Spring to Proust")
provenance_entry.part = acquisition

# What object was transferred: The "Spring" painting
acquisition.transferred_title_of = model.HumanMadeObject(
    ident="spring",
    label="Spring"
)

# Who transferred ownership from: Manet (seller)
acquisition.transferred_title_from = model.Person(
    ident="manet",
    label="Manet"
)

# Who transferred ownership to: Proust (buyer)
acquisition.transferred_title_to = model.Person(
    ident="proust",
    label="Proust"
)

# Payment event - money exchanged
payment = model.Payment(label="3000 Francs to Manet")
provenance_entry.part = payment

# Who paid: Proust (buyer)
payment.paid_from = model.Person(ident="proust", label="Proust")

# Who received payment: Manet (seller)
payment.paid_to = model.Person(ident="manet", label="Manet")

# What amount was paid: 3,000 francs
amount = model.MonetaryAmount()
amount.currency = vocab.instances['fr francs']
amount.value = 3000
payment.paid_amount = amount

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(provenance_entry, compact=False))


 
### Payment for Services

Beyond simply paying the previous owner for the object, there are many other reasons why money might change hands that are relevant to describe. For example, it might be known how much commission went to an auction house or for the sale of objects by consignment, particularly if this comes from the stock books or records of the company. Artists might be paid a commission in advance for the production of an object, or agents might be paid a commission for finding and purchasing objects on behalf of the new owner. This is often not recorded in detail, however the pattern is provided for situations when it is known.

__Example:__

A sculpture is purchased with $900 going to the seller, and $100 going to the auction house.



In [ ]:
# 步骤 1: 导入 cromulent 库
from cromulent import model, vocab

# 步骤 2: 配置工厂设置
model.factory.auto_assign_id = False
vocab.add_attribute_assignment_check()

# 步骤 3: 创建主对象 (venance entry with commission)
# 行为者: Unknown buyer, Unknown seller, Auction House (commission recipient)
# 对象: ProvenanceEntry representing a purchase with agent commission
# 目的: To document multiple payment flows in a single transaction
provenance_entry = vocab.ProvenanceEntry(
    ident="fixme/1",
    label="Purchase of Sculpture with Commission"
)

# 步骤 4: 创建相关对象和关系
# 行为者: The parties involved
seller = model.Person(label="Seller")
buyer = model.Person(label="Buyer")
house = vocab.AuctionHouseOrg(label="Auction House")

# First payment: Buyer pays seller for the artwork
payment1 = model.Payment()
payment1.paid_from = buyer
payment1.paid_to = seller

# What amount was paid to seller: $900
amount1 = model.MonetaryAmount()
amount1.value = 900
amount1.currency = vocab.instances['us dollars']
payment1.paid_amount = amount1
provenance_entry.part = payment1

# Second payment: Buyer pays commission to auction house
payment2 = vocab.CommissionPayment()
payment2.paid_from = buyer
payment2.paid_to = house

# What amount was paid as commission: $100
amount2 = model.MonetaryAmount()
amount2.value = 100
amount2.currency = vocab.instances['us dollars']
payment2.paid_amount = amount2
provenance_entry.part = payment2

# 步骤 5: 显示生成的 JSON-LD
print(model.factory.toString(provenance_entry, compact=False))